In [1]:
# ============================================================
# 📦 EDM2 SETUP
# ============================================================

!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow matplotlib --quiet

import sys, os
sys.path.append("/kaggle/working/edm2")

print("✅ EDM2 setup complete.")

# Create output folder for Block 1
BLOCK1_DIR = "/kaggle/working/block1_outputs"
os.makedirs(BLOCK1_DIR, exist_ok=True)

# ============================================================
# 📦 IMPORTS
# ============================================================

import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pickle

# ============================================================
# 📦 NOISE SCHEDULES (num = 20)
# ============================================================

def karras_rho_schedule(num=20, sigma_min=0.002, sigma_max=80, rho=7):
    ramp = torch.linspace(0, 1, num)
    return (sigma_max**(1/rho) + ramp * (sigma_min**(1/rho) - sigma_max**(1/rho)))**rho

def linear_schedule(num=20, sigma_min=0.002, sigma_max=80):
    return torch.linspace(sigma_max, sigma_min, num)

def quadratic_schedule(num=20, sigma_min=0.002, sigma_max=80):
    t = torch.linspace(0, 1, num)
    return sigma_max - (sigma_max - sigma_min) * (t**2)

def cosine_schedule(num=20, sigma_min=0.002, sigma_max=80):
    t = torch.linspace(0, 1, num)
    return sigma_min + (sigma_max - sigma_min) * 0.5 * (1 + torch.cos(torch.pi * t))

def logarithmic_schedule(num=20, sigma_min=0.002, sigma_max=80):
    return torch.logspace(np.log10(sigma_max), np.log10(sigma_min), num)

# ============================================================
# 📦 MODEL PATHS
# ============================================================

MODEL_PATHS = {
    "karras":      "/kaggle/input/all-uc-trained-noise-models/pytorch/default/1/network-snapshot-0002097-0.100_karrasrho_UC.pkl",
    "linear":      "/kaggle/input/all-uc-trained-noise-models/pytorch/default/1/network-snapshot-0002097-0.100_linear_UC.pkl",
    "cosine":      "/kaggle/input/all-uc-trained-noise-models/pytorch/default/1/network-snapshot-0002097-0.100_cosine_UC.pkl",
    "logarithmic": "/kaggle/input/all-uc-trained-noise-models/pytorch/default/1/network-snapshot-0002097-0.100_loagarithmic_UC.pkl",
    "quadratic":   "/kaggle/input/all-uc-trained-noise-models/pytorch/default/1/network-snapshot-0002097-0.100_quadrattic_UC.pkl",
}

SCHEDULES = {
    "karras": karras_rho_schedule,
    "linear": linear_schedule,
    "quadratic": quadratic_schedule,
    "cosine": cosine_schedule,
    "logarithmic": logarithmic_schedule,
}

# ============================================================
# 📦 IMAGE LIST (5 IMAGES)
# ============================================================

IMAGE_PATHS = [
    "/kaggle/input/celeva-64x64-dataset/celeba64/test/182644.jpg",
    "/kaggle/input/celeva-64x64-dataset/celeba64/test/182650.jpg",
    "/kaggle/input/celeva-64x64-dataset/celeba64/test/182661.jpg",
    "/kaggle/input/celeva-64x64-dataset/celeba64/test/182669.jpg",
    "/kaggle/input/celeva-64x64-dataset/celeba64/test/182755.jpg",
]

# ============================================================
# 📦 DENOISE STEP
# ============================================================

def denoise_step(net, x_noisy, sigma):
    sigma_tensor = torch.tensor([sigma], device="cuda").reshape(1,1,1,1)
    D = net(x_noisy, sigma_tensor)
    return x_noisy - sigma_tensor * D

# ============================================================
# 📦 PROCESS MODEL ( SAME NOISE FOR ALL σ STEPS )
# ============================================================

def process_model(model_name, model_path, schedule_func, x0, num_steps=20):
    print(f"\n=== Processing {model_name.upper()} ===")

    with open(model_path, "rb") as f:
        data = pickle.load(f)
    net = data["ema"].to("cuda").eval()

    schedule = schedule_func(num_steps).cuda()

    # ⭐ SAME NOISE FOR ALL SIGMA VALUES
    base_noise = torch.randn_like(x0)

    noisy_imgs, denoised_imgs, sigmas = [], [], []

    for sigma in schedule:
        s = sigma.item()
        sigmas.append(s)

        x_noisy = x0 + sigma * base_noise
        x_denoised = denoise_step(net, x_noisy, s)

        noisy_imgs.append(x_noisy[0].cpu())
        denoised_imgs.append(x_denoised[0].cpu())

    return sigmas, noisy_imgs, denoised_imgs

# ============================================================
# 📦 MAIN LOOP — Save to block1_outputs/
# ============================================================

for img_path in IMAGE_PATHS:

    img_id = os.path.basename(img_path).split(".")[0]

    img = Image.open(img_path).convert("RGB")
    tf = T.Compose([T.Resize((64,64)), T.ToTensor()])
    x0 = tf(img).unsqueeze(0).cuda()

    results = {}

    for name, path in MODEL_PATHS.items():
        sigmas, noisy, den = process_model(name, path, SCHEDULES[name], x0)
        results[name] = (sigmas, noisy, den)

        # Save sigma plot into block1 directory
        plt.figure(figsize=(10,4))
        plt.plot(sigmas, marker="o")
        plt.grid(True)
        plt.xlabel("Step Index"); plt.ylabel("Sigma Value")
        plt.title(f"Sigma Progression — {name}")
        plt.savefig(f"{BLOCK1_DIR}/{img_id}_{name}_sigma_plot.png")
        plt.close()

        # Save noisy + denoised grid
        fig, axes = plt.subplots(2, len(sigmas), figsize=(3*len(sigmas), 6))
        for i in range(len(sigmas)):
            axes[0, i].imshow(noisy[i].permute(1,2,0).clip(0,1))
            axes[1, i].imshow(den[i].permute(1,2,0).clip(0,1))
            axes[0, i].set_title(f"{name}\nσ={round(sigmas[i],3)}")
            axes[0, i].axis("off"); axes[1, i].axis("off")

        plt.tight_layout()
        plt.savefig(f"{BLOCK1_DIR}/{img_id}_{name}_sigma_grid.png")
        plt.close()

    # Combined comparison grid
    num_steps = len(sigmas)
    fig, axes = plt.subplots(5, num_steps, figsize=(3*num_steps, 18))

    for row_idx, (name, (sigmas, noisy_imgs, _)) in enumerate(results.items()):
        for col in range(num_steps):
            axes[row_idx, col].imshow(noisy_imgs[col].permute(1,2,0).clip(0,1))
            axes[row_idx, col].set_title(f"{name}\nσ={round(sigmas[col],3)}")
            axes[row_idx, col].axis("off")

    plt.tight_layout()
    plt.savefig(f"{BLOCK1_DIR}/{img_id}_all_schedules_comparison.png")
    plt.close()


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 10.34 MiB/s, done.
Resolving deltas: 100% (24/24), done.
✅ EDM2 setup complete.

=== Processing KARRAS ===

=== Processing LINEAR ===

=== Processing COSINE ===

=== Processing LOGARITHMIC ===

=== Processing QUADRATIC ===

=== Processing KARRAS ===

=== Processing LINEAR ===

=== Processing COSINE ===

=== Processing LOGARITHMIC ===

=== Processing QUADRATIC ===

=== Processing KARRAS ===

=== Processing LINEAR ===

=== Processing COSINE ===

=== Processing LOGARITHMIC ===

=== Processing QUADRATIC ===

=== Processing KARRAS ===

=== Processing LINEAR ===

=== Processing COSINE ===

=== Processing LOGARITHMIC ===

=== Processing QUADRATIC ===

=== Processing KARRAS ===

=== Processing LINEAR ===

=

In [2]:
# ============================================================
# 📦 BLOCK 2 — DENOISING AT SAME SIGMA ACROSS ALL SCHEDULES
# ============================================================

# Create folder for Block 2 outputs
BLOCK2_DIR = "/kaggle/working/block2_outputs"
os.makedirs(BLOCK2_DIR, exist_ok=True)

sigma_small   = torch.linspace(0.002, 0.1, 10)
sigma_medium  = torch.linspace(0.1,   1.0, 5)
sigma_big     = torch.tensor([1.5, 2, 2.5, 3, 5, 10, 20, 40, 60, 80])
SIGMA_LIST = torch.cat([sigma_small, sigma_medium, sigma_big]).cuda()
NUM_SIGMAS = len(SIGMA_LIST)

def denoise_across_fixed_sigmas(model_path, x0, sigma_list):
    with open(model_path, "rb") as f:
        net = pickle.load(f)["ema"].to("cuda").eval()

    base_noise = torch.randn_like(x0)

    imgs = []
    for sigma in sigma_list:
        s = sigma.item()
        x_noisy = x0 + sigma * base_noise
        x_d = denoise_step(net, x_noisy, s)
        imgs.append(x_d[0].cpu())
    return imgs

for img_path in IMAGE_PATHS:

    img_id = os.path.basename(img_path).split(".")[0]

    img = Image.open(img_path).convert("RGB")
    tf = T.Compose([T.Resize((64,64)), T.ToTensor()])
    x0 = tf(img).unsqueeze(0).cuda()

    denoise_results = {}

    for name, path in MODEL_PATHS.items():
        denoise_results[name] = denoise_across_fixed_sigmas(path, x0, SIGMA_LIST)

    fig, axes = plt.subplots(5, NUM_SIGMAS, figsize=(3*NUM_SIGMAS, 18))

    for row_idx, (name, imgs) in enumerate(denoise_results.items()):
        for col_idx, img_d in enumerate(imgs):
            axes[row_idx, col_idx].imshow(img_d.permute(1,2,0).clip(0,1))
            sigma_val = float(SIGMA_LIST[col_idx].cpu())
            axes[row_idx, col_idx].set_title(f"{name}\nσ={sigma_val:.3f}")
            axes[row_idx, col_idx].axis("off")

    plt.tight_layout()
    plt.savefig(f"{BLOCK2_DIR}/{img_id}_denoising_same_sigma.png")
    plt.close()
